In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X,y = data.drop('target',axis=1),data['target']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [6]:
import torch
import torch.nn as nn

In [7]:
import numpy as np

In [8]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [9]:
X_train.shape

torch.Size([227, 13])

In [10]:
X_test.shape

torch.Size([76, 13])

In [11]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,16)
        self.fc2 = nn.Linear(16,32)
        self.fc3 = nn.Linear(32,64)
        self.fc4 = nn.Linear(64,128)
        self.fc5 = nn.Linear(128,256)
        self.fc6 = nn.Linear(256,128)
        self.fc7 = nn.Linear(128,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
model = Test_Model().to(device)

In [17]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)

In [18]:
criterion = nn.L1Loss()

In [19]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [20]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [21]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [22]:
import wandb

In [23]:
from tqdm import tqdm

In [24]:
EPOCHS = 100

In [25]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [26]:
preds_new = []

In [27]:
for pred in preds:
    pred = pred[0]
    if pred > 1.0:
        preds_new.append(0.0)
    elif pred == 1.0:
        preds_new.append(1.0)

In [28]:
preds

tensor([0.4096, 0.3493, 0.8591, 0.4277, 0.5052, 0.6855, 0.9639, 0.7996, 0.8606,
        0.6374, 0.9308, 0.8384, 0.3470, 0.5556, 0.5994, 0.6610, 0.9334, 0.9535,
        0.5172, 0.7349, 0.9048, 0.7733, 0.7500, 0.3497, 0.5038, 0.8552, 0.5496,
        0.8641, 0.3933, 0.9067, 0.5115, 0.8678, 0.7530, 0.8220, 0.9038, 0.9120,
        0.9466, 0.9036, 0.4103, 0.8073, 0.9215, 0.7751, 0.4525, 0.6009, 0.3703,
        0.6452, 0.8186, 0.9081, 0.9115, 0.8163, 0.8486, 0.8930, 0.8114, 0.5770,
        0.8906, 0.9014, 0.8227, 0.7856, 0.3503, 0.7932, 0.8805, 0.8242, 0.7974,
        0.7253, 0.8739, 0.9185, 0.3253, 0.3037, 0.3066, 0.7537, 0.7151, 0.4396,
        0.8515, 0.8797, 0.9102, 0.3747, 0.9024, 0.6632, 0.3536, 0.6017, 0.6532,
        0.7562, 0.9098, 0.8386, 0.9154, 0.6110, 0.3568, 0.9155, 0.6125, 0.7212,
        0.9497, 0.7027, 0.3311, 0.6598, 0.5885, 0.6487, 0.8909, 0.8574, 0.9196,
        0.6612, 0.8666, 0.9234, 0.9328, 0.7401, 0.9191, 0.7548, 0.6314, 0.9475,
        0.9527, 0.5859, 0.8448, 0.6429, 

In [29]:
for pred in preds:
#     pred = pred[0]
    if pred > 1.0:
        preds_new.append(0.0)
    elif pred == 1.0:
        preds_new.append(1.0)

In [30]:
preds_new

[]

In [31]:
for pred in preds:
#     pred = pred[0]
    print(pred)
    if pred > 1.0:
        preds_new.append(0.0)
    elif pred == 1.0:
        preds_new.append(1.0)

In [32]:
for pred in preds:
#     pred = pred[0]
    print(pred>1.0)
    if pred > 1.0:
        preds_new.append(0.0)
    elif pred == 1.0:
        preds_new.append(1.0)

In [33]:
for pred in preds:
#     pred = pred[0]
    print(pred>0.99)
    if pred > 1.0:
        preds_new.append(0.0)
    elif pred == 1.0:
        preds_new.append(1.0)

In [34]:
for pred in preds:
#     pred = pred[0]
    print(pred)
    if pred > 1.0:
        preds_new.append(0.0)
    elif pred == 1.0:
        preds_new.append(1.0)

In [35]:
torch.round(preds)

tensor([0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
        1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
        1., 0., 1., 1., 0., 1., 0., 1., 

In [36]:
y_train[:5]

tensor([0., 1., 0., 0., 0.], device='cuda:0')

In [37]:
torch.round(preds)[:5]

tensor([0., 0., 1., 0., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [38]:
torch.round(preds)[:25]

tensor([0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [39]:
y_train[:25]

tensor([0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 1., 1., 0., 1., 0., 1.], device='cuda:0')

In [40]:
torch.round(preds)[:12]

tensor([0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [41]:
y_train[:12]

tensor([0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.], device='cuda:0')

In [42]:
import pandas as pd

In [43]:
data = pd.read_csv('./data.csv')

In [44]:
X,y = data.drop('target',axis=1),data['target']

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [47]:
import torch
import torch.nn as nn

In [48]:
import numpy as np

In [49]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [50]:
X_train.shape

torch.Size([227, 13])

In [51]:
X_test.shape

torch.Size([76, 13])

In [52]:
y_train.shape

torch.Size([227])

In [53]:
y_test.shape

torch.Size([76])

In [54]:
import torch.nn.functional as F

In [55]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,128)
        self.fc4 = nn.Linear(128,256)
        self.fc5 = nn.Linear(256,512)
        self.fc6 = nn.Linear(512,256)
        self.fc7 = nn.Linear(256,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [56]:
device = torch.device('cuda')

In [57]:
model = Test_Model().to(device)

In [58]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)

In [59]:
criterion = nn.L1Loss()

In [60]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [61]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [62]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [63]:
import wandb

In [64]:
from tqdm import tqdm

In [65]:
EPOCHS = 100

In [66]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()